In [1]:
from sklearn.neighbors import KDTree
import numpy as np
import random
import dask.array as da
import napari
import pandas as pd
import zarr

%gui qt

coord_path = 'C:/Users/camer/OneDrive - Allen Institute/Documents/Unet/0539050069-0035_r.csv'

coord = np.loadtxt(coord_path, skiprows=1, delimiter=',')

In [2]:
z = zarr.open('C:/Users/camer/OneDrive - Allen Institute/Documents/Unet/0539050069-0035.zarr')
for i in range(z.shape[0] // 260):
    z[(i*260)-1:(i*260)+1, :, 0] = 5000
for i in range (z.shape[1] // 260):
    z[:, (i*260)-1:(i*260)+1, 0] = 5000
image = da.from_zarr(z)

In [28]:
# zarr_img_path = 'C:/Users/camer/OneDrive - Allen Institute/Documents/Unet/0539050069-0035.zarr'
# image = da.from_zarr(zarr_img_path)

In [3]:
coord_df = pd.DataFrame(coord)
coord_df.columns = ['y', 'x', 'size']
size_over_50 = coord_df['size'] >= 50
coord_df = coord_df[size_over_50]
distance_from_2048 = 50
before_vert = coord_df['x'] % 260 <= distance_from_2048
after_vert = coord_df['x'] % 260 >= (260 - distance_from_2048)
before_horiz = coord_df['y'] % 260 <= distance_from_2048
after_horiz = coord_df['y'] % 260 >= (260 - distance_from_2048)
coord_df = coord_df[(before_horiz | after_horiz) | (before_vert | after_vert)]
coord_s50 = coord_df.to_numpy()
print(coord_s50.shape)

(9509, 3)


In [4]:
kdt = KDTree(coord_s50[:,:2], leaf_size=30, metric='euclidean')

close_points = []
close_set =[]
indices_to_drop = []

close = kdt.query_radius(coord_s50[:,:2], r=15) 
for i in close:
    if i.size > 1:
        close_points.append(list(i))
for i in close_points:
    if not i in close_set:
        close_set.append(i)
print("Close points are: ", str(close_set))
for i in close_set:
    while len(i)>1:
        index = random.randrange(len(i))
        indices_to_drop.append(i[index])
        del i[index]

print(str(len(indices_to_drop)),"points to be removed:", str(indices_to_drop))
# print("These points are located at:")
# np.set_printoptions(suppress=True)
# for i in indices_to_drop:
#     print(coord[indices_to_drop])
# np.set_printoptions(suppress=False)
coord_s50_clean = np.array([i for j, i in enumerate(coord_s50) if j not in indices_to_drop])
coord_s50_removed = np.array([i for j, i in enumerate(coord_s50) if j in indices_to_drop])

print(str(coord_s50.shape[0]-coord_s50_clean.shape[0]), "points removed")

Close points are:  [[1122, 80], [647, 782], [1652, 774], [2066, 962], [986, 1136], [1121, 2409], [1279, 2564], [1373, 2733], [1752, 1818], [1851, 3088], [1997, 2144], [2063, 3181], [2064, 3180], [2216, 3265], [2372, 2377], [2561, 3754], [2741, 2819], [3416, 4790], [3541, 4948], [3886, 3764], [4126, 4129], [5664, 4222], [4465, 4307], [4469, 5795], [4652, 4515], [6007, 4525], [4526, 6006], [4661, 6154], [4772, 4899], [6435, 4947], [5225, 5226], [5409, 5484], [5415, 5514], [5728, 7127], [5869, 5755], [6135, 5979], [6260, 7473], [6539, 6670], [6559, 7663], [6735, 6737], [6942, 7977], [7981, 7093], [7355, 7493], [7544, 7609], [7857, 7856], [8033, 8068], [8685, 8109], [8143, 8201], [9070, 8463], [8503, 8569], [9032, 8979], [9012, 9075]]
52 points to be removed: [80, 782, 774, 2066, 1136, 1121, 1279, 2733, 1752, 1851, 1997, 3181, 3180, 2216, 2372, 3754, 2741, 3416, 3541, 3886, 4129, 4222, 4465, 4469, 4652, 6007, 6006, 4661, 4772, 6435, 5225, 5409, 5514, 7127, 5869, 6135, 6260, 6539, 6559, 673

In [5]:
viewer = napari.view_image(image[:, :, 0], name='slice')
viewer.add_points(coord_s50_removed[:, :2], size=20, face_color='yellow')
viewer.add_points(coord_s50_clean[:, :2], size=10, face_color='red')

C:\Users\camer\anaconda3\envs\napari\lib\site-packages\napari\_vispy\vispy_image_layer.py:198: UserWarning: data shape (30000, 40000) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


<Points layer 'Points [1]' at 0x16c03244a60>

In [276]:
np.savetxt('/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/csv/0539050069-0035_r_clean.csv', coord_clean, delimiter=",")

np.savetxt('/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/csv/0539050069-0035_r_removed.csv', coord_removed, delimiter=",")